In [ ]:
import pandas as pd
import re

Set the log files paths

In [ ]:
connection_logs_file_path = "connection_logs.txt"
cloudquery_logs_file_path = "cloudquery.log"

Set Patterns dictionnary for connection logs

In [ ]:
patterns_dict = {
    "interface": ["^eth0$", "^lo$"],
    "direction": ["^In$", "^Out$"],
    "protocol": ["^IP$", "^IP6$"],
    "timestamp": [r"^\d{2}:\d{2}:\d{2}\.\d{6}$"],
    "tcp_flags": [r"^[FSRPAUEC]+$"],
    "ipv4": [r"^(\d{1,3}\.){3}\d{1,3}.\d{1,5}:?$"],
    "ipv6": [r"^(?:[0-9a-fA-F]{0,4}:){2,7}[0-9a-fA-F]{0,4}$"],
}

Define functions to parse logs

In [ ]:
def parse_log_chunk(chunk):
  data_dict = {}
  for line in chunk:
    strings = line.split()
    num_strings = len(strings)
    if num_strings not in data_dict:
      data_dict[num_strings] = []
    data_dict[num_strings].append(strings)
  return {k: pd.DataFrame(v) for k, v in data_dict.items()}


In [ ]:
def parse_log_file(file_path, chunk_size=50000, max_chunks=None, display_chunk_number=False):
  chunks = pd.read_csv(file_path, chunksize=chunk_size, delimiter="\t", header=None)
  dataframes_dict = {}
  for i, chunk in enumerate(chunks, start=1):
    if max_chunks is not None and i > max_chunks:
      break
    if display_chunk_number:
      print(f'Processing chunk {i}')
    parsed_chunk_dict = parse_log_chunk(chunk[0])
    for num_strings, df in parsed_chunk_dict.items():
      if num_strings not in dataframes_dict:
        dataframes_dict[num_strings] = df
      else:
        dataframes_dict[num_strings] = pd.concat([dataframes_dict[num_strings], df])
  return dataframes_dict

Define functions to match patterns

In [ ]:
def matches_any_pattern(dataframe, column, patterns):
    """Check if any of the patterns match the values in the column of the dataframe."""
    return dataframe[column].apply(
        lambda x: any(re.match(pattern, x) for pattern in patterns)
    )

In [ ]:
def filter_connection_logs(dataframe, patterns_dict):
    """Filter out rows that do not match the expected patterns."""
    # Check if column 0 is a timestamp
    col0_eval = matches_any_pattern(dataframe, 0, patterns_dict["timestamp"])

    # Check if column 1 is an interface
    col1_eval = matches_any_pattern(dataframe, 1, patterns_dict["interface"])

    # Check if column 4 is an IP address
    col4_eval = matches_any_pattern(dataframe, 4, patterns_dict["ipv4"]) | matches_any_pattern(dataframe, 4, patterns_dict["ipv6"])

    # Combine the conditions
    conditions_met = col0_eval & col1_eval & col4_eval

    df_cleaned = dataframe[conditions_met]
    return df_cleaned

Define functions to filter the rows and columns based on the patterns

In [90]:
def validate_and_rename_columns(df, patterns_dict):
    df_copy = df.copy()
    expected_columns = ["timestamp", "interface", None, "protocol", "ip", None, "ip"]
    ip_pattern_name = ["source", "destination"]
    ip_count = 0
    for i, column in enumerate(df.columns):
        if i < len(expected_columns):
            expected_pattern_name = expected_columns[i]
        else:
            expected_pattern_name = None

        if expected_pattern_name is None:
            continue

        if expected_pattern_name == "ip":
            ipv4_patterns = patterns_dict["ipv4"]
            ipv6_patterns = patterns_dict["ipv6"]
            pattern = re.compile("|".join(ipv4_patterns + ipv6_patterns))
            expected_pattern_name = ip_pattern_name[ip_count]
            ip_count += 1
        else:
            expected_patterns = patterns_dict[expected_pattern_name]
            pattern = re.compile("|".join(expected_patterns))

        mask = df_copy[column].apply(lambda x: bool(pattern.match(str(x))))
        df_copy = df_copy[mask]

        df_copy = df_copy.rename(columns={column: expected_pattern_name})

    return df_copy


In [120]:
def format_ip_address(ip_address, ip_type):
  if ip_type == "IPv4":
    ip_parts = ip_address.split(".")
    port = ip_parts[-1]
    if port.endswith(":"):
      port = port[:-1]
    return (".".join(ip_parts[:-1]), port)
  elif ip_type == "IPv6":
    return (ip_address, None)
  else:
    return (None, None)

In [164]:
def format_connection_logs(df_dict_cleaned, patterns_dict):
    df_list = []
    for key, df in df_dict_cleaned.items():
        print(f"Formatting df with key: {key}")
        df_filtered = validate_and_rename_columns(df, patterns_dict)
        print(f"Dataframe has {df_filtered.shape[0]} rows and {df_filtered.shape[1]} columns")
        direction_mask = matches_any_pattern(df_filtered, 2, patterns_dict["direction"])
        df_filtered.loc[direction_mask, "direction"] = df_filtered.loc[direction_mask, 2]
        df_filtered.loc[~direction_mask, "flags"] = df_filtered.loc[~direction_mask, 2]

        # Create masks for source and destination strings that match IPv4 or IPv6 patterns
        source_ipv4_mask = matches_any_pattern(
            df_filtered, "source", patterns_dict["ipv4"]
        )
        source_ipv6_mask = matches_any_pattern(
            df_filtered, "source", patterns_dict["ipv6"]
        )
        destination_ipv4_mask = matches_any_pattern(
            df_filtered, "destination", patterns_dict["ipv4"]
        )
        destination_ipv6_mask = matches_any_pattern(
            df_filtered, "destination", patterns_dict["ipv6"]
        )

        # Assign type based on masks
        df_filtered.loc[source_ipv4_mask, "source_type"] = "IPv4"
        df_filtered.loc[source_ipv6_mask, "source_type"] = "IPv6"
        df_filtered.loc[destination_ipv4_mask, "destination_type"] = "IPv4"
        df_filtered.loc[destination_ipv6_mask, "destination_type"] = "IPv6"

        # Split source and destination strings based on their type
        df_filtered["source_ip"], df_filtered["source_port"] = zip(
            *df_filtered.apply(
                lambda row: format_ip_address(row["source"], row["source_type"]),
                axis=1,
            )
        )
        df_filtered["destination_ip"], df_filtered["destination_port"] = zip(
            *df_filtered.apply(
                lambda row: format_ip_address(row["destination"], row["destination_type"]),
                axis=1,
            )
        )
        df_list.append(df_filtered)
    df_formatted = pd.concat(df_list, ignore_index=True)
    columns_to_drop = [
        "source",
        "destination",
    ]
    for column in df_formatted.columns:
        if str(column).isdigit() and not column in [9,10]:
            columns_to_drop.append(column)
    df_formatted = df_formatted.drop(columns=columns_to_drop)
    print(f"Final dataframe has {df_formatted.shape[0]} rows and {df_formatted.shape[1]} columns")
    return df_formatted

Parse the connection logs (max 60 chunks for now otherwise the kernel will crash)

In [ ]:
connection_logs_dataframes_dict = parse_log_file(connection_logs_file_path, max_chunks=50)

In [ ]:
print(connection_logs_dataframes_dict.keys())
for key, value in connection_logs_dataframes_dict.items():
    print(f"Key: {key}, Value: {value.head()}")

Cleanup the data by filtering out the rows and columns that don't match the patterns soughts

In [ ]:

df_dict_cleaned = {}
for key, df in connection_logs_dataframes_dict.items():
    print(f"Processing dataframe with {key} strings")
    # print(df.head())
    df_cleaned = filter_connection_logs(df, patterns_dict)
    if len(df_cleaned) > 500:
        df_dict_cleaned[key] = df_cleaned
        print(f"Dataframe with {key} strings has {len(df_cleaned)} rows")
    # print(df_cleaned.head())

In [86]:
print(df_dict_cleaned)

{10:                   0     1   2   3                 4  5                     6      7      8      9
4   20:04:54.430117  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.56860:  39650  0/1/1  (137)
5   20:04:54.431064  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.41260:  64911  0/1/1  (153)
6   20:04:54.440611  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.52596:  45912  0/1/1  (139)
7   20:04:54.446585  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.49054:  18432  0/1/1  (135)
8   20:04:54.452936  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.35735:  43373  0/1/1  (143)
..              ...   ...  ..  ..               ... ..                   ...    ...    ...    ...
25  20:05:19.203917  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.56272:  25442  0/1/1  (138)
26  20:05:19.206701  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.42234:  48854  0/1/1  (147)
27  20:05:19.210723  eth0  In  IP  168.63.129.16.53  >  192.168.144.3.36803:  41997  0/1/1  (142)
28  20:05:19.21

Printing the filtered data

In [ ]:
# Set the display width to a large value
pd.set_option("display.width", 1000)

# Set the maximum columns to a large value
pd.set_option("display.max_columns", 50)

# Sort the items in df_dict_cleaned by the value of item[0]
sorted_items = sorted(df_dict_cleaned.items(), key=lambda item: item[0])

for item in sorted_items:
    print(f"Dataframe with {item[0]} strings:")
    print(item[1].head())
    print("")


In [57]:
print(df_dict_cleaned.keys())

dict_keys([10, 12, 23, 25, 21, 15, 17, 19])


Test the format connection logs based on the desired final output

In [160]:
# Set the display width to a large value
pd.set_option("display.width", 1000)

# Set the maximum columns to a large value
pd.set_option("display.max_columns", 50)
# Get the first two keys from df_dict_cleaned
first_two_keys = list(df_dict_cleaned.keys())[:3]

# Create a new dictionary with only the first two dataframes
df_dict_cleaned_first_two = {key: df_dict_cleaned[key] for key in first_two_keys}

# Create a new dictionary with only the specific dataframe
df_dict_cleaned_specific = {12: df_dict_cleaned[12]}

df_test = format_connection_logs(df_dict_cleaned_specific, patterns_dict)
# print(df_test.head(300))

Formatting df with key: 12
Dataframe has 958 rows and 12 columns
Columns to drop: ['source', 'destination', 2, 5, 7, 8, 11]
Final dataframe has 958 rows and 13 columns


In [161]:
print(df_test.head(10))

         timestamp interface protocol              9                                           10 direction flags source_type destination_type                  source_ip source_port destination_ip destination_port
0  20:02:05.732749      eth0      IP6  solicitation,                                       length       NaN     M        IPv6             IPv6   fe80::f49f:17ff:fe3f:587        None       ff02::2:             None
1  20:03:26.372765      eth0      IP6  solicitation,                                       length       NaN     M        IPv6             IPv6  fe80::90b1:c8ff:fe28:9472        None       ff02::2:             None
2  20:03:33.540746      eth0      IP6  solicitation,                                       length       NaN     M        IPv6             IPv6  fe80::90b1:c8ff:fe28:9472        None       ff02::2:             None
3  20:03:48.388760      eth0      IP6  solicitation,                                       length       NaN     M        IPv6             IPv6  

Final formatting of the connection logs

In [162]:

df_final = format_connection_logs(df_dict_cleaned, patterns_dict)

Formatting df with key: 10
Dataframe has 600 rows and 10 columns
Formatting df with key: 12
Dataframe has 958 rows and 12 columns
Formatting df with key: 23
Dataframe has 840552 rows and 23 columns
Formatting df with key: 25
Dataframe has 14809 rows and 25 columns
Formatting df with key: 21
Dataframe has 526382 rows and 21 columns
Formatting df with key: 15
Dataframe has 489015 rows and 15 columns
Formatting df with key: 17
Dataframe has 598597 rows and 17 columns
Formatting df with key: 19
Dataframe has 28942 rows and 19 columns
Columns to drop: ['source', 'destination', 2, 5, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Final dataframe has 2499855 rows and 13 columns


In [166]:
print(df_final.head(100))

          timestamp interface protocol      9 direction flags source_type destination_type      source_ip source_port destination_ip destination_port   10
0   20:04:54.430117      eth0       IP  (137)        In   NaN        IPv4             IPv4  168.63.129.16          53  192.168.144.3            56860  NaN
1   20:04:54.431064      eth0       IP  (153)        In   NaN        IPv4             IPv4  168.63.129.16          53  192.168.144.3            41260  NaN
2   20:04:54.440611      eth0       IP  (139)        In   NaN        IPv4             IPv4  168.63.129.16          53  192.168.144.3            52596  NaN
3   20:04:54.446585      eth0       IP  (135)        In   NaN        IPv4             IPv4  168.63.129.16          53  192.168.144.3            49054  NaN
4   20:04:54.452936      eth0       IP  (143)        In   NaN        IPv4             IPv4  168.63.129.16          53  192.168.144.3            35735  NaN
..              ...       ...      ...    ...       ...   ...         

Data Analysis part (CIDR blocks)

In [185]:
# Identify all unique destination IP addresses in a single list where the direction is NOT "In" and the protocol is "IP"
unique_destination_ips = df_final[(df_final["direction"] != "In") & (df_final["protocol"] == "IP")]["destination_ip"].unique()
print(len(unique_destination_ips))
print(unique_destination_ips)

2122
['168.63.129.16' '142.251.163.121' '172.253.115.121' ... '52.217.85.112'
 '52.217.81.6' '52.217.71.120']


In [170]:
import ipaddress
import netaddr

# Convert the list of IP addresses to ipaddress.IPv4Address objects
ip_objects = [ipaddress.ip_address(ip) for ip in unique_destination_ips]

# Sort the list of IP addresses
ip_objects.sort()

# Convert the sorted list of IP addresses back to strings
sorted_ips = [str(ip) for ip in ip_objects]

# Use netaddr.cidr_merge to merge the list of IP addresses into CIDR blocks
cidr_blocks = netaddr.cidr_merge(sorted_ips)

# Print the CIDR blocks
for block in cidr_blocks:
    print(block)

3.5.2.113/32
3.5.7.163/32
3.5.7.201/32
3.5.19.116/32
3.5.76.22/32
3.5.76.107/32
3.5.76.132/32
3.5.76.178/32
3.5.77.190/32
3.5.78.12/32
3.5.78.17/32
3.5.78.105/32
3.5.78.128/31
3.5.79.111/32
3.5.79.115/32
3.5.79.171/32
3.5.80.102/32
3.5.80.138/32
3.5.81.119/32
3.5.81.128/32
3.5.81.161/32
3.5.81.175/32
3.5.81.201/32
3.5.82.1/32
3.5.82.125/32
3.5.82.129/32
3.5.82.132/32
3.5.82.161/32
3.5.83.13/32
3.5.83.152/32
3.5.83.169/32
3.5.83.182/32
3.5.83.213/32
3.5.84.16/32
3.5.84.174/32
3.5.84.176/32
3.5.84.221/32
3.5.85.98/32
3.5.85.147/32
3.5.85.169/32
3.5.252.61/32
3.5.252.154/32
3.5.252.167/32
3.5.253.66/32
3.5.253.129/32
3.5.253.205/32
3.5.254.175/32
3.5.254.220/32
3.83.170.151/32
3.83.170.158/32
3.83.170.183/32
3.83.170.226/32
3.83.171.158/32
3.86.133.236/32
3.86.168.174/32
3.91.171.215/32
3.91.171.219/32
3.91.171.220/32
3.91.171.231/32
3.91.171.234/32
3.91.171.239/32
3.91.171.240/32
3.91.171.242/32
3.91.171.244/32
3.91.171.246/32
3.91.171.252/31
3.91.171.254/32
3.92.117.191/32
3.94.174.123/

In [179]:
import json
# load the Github endpoint json file as an object
with open("github.json") as f:
    github_endpoints = json.load(f)

# keep only the following keys
print(github_endpoints.keys())

github_cidr_blocks = []
github_keys = ["hooks", "pages", "git", "api", "packages"]
for key in github_endpoints.keys():
    if key not in github_keys:
        continue
    # for endpoint in github_endpoints[key]:
    #     if "cidr" in endpoint:
    github_cidr_blocks.extend(github_endpoints[key])

print(github_cidr_blocks)


dict_keys(['verifiable_password_authentication', 'ssh_key_fingerprints', 'ssh_keys', 'hooks', 'web', 'api', 'git', 'github_enterprise_importer', 'packages', 'pages', 'importer', 'actions', 'dependabot', 'domains'])
['192.30.252.0/22', '185.199.108.0/22', '140.82.112.0/20', '143.55.64.0/20', '2a0a:a440::/29', '2606:50c0::/32', '192.30.252.0/22', '185.199.108.0/22', '140.82.112.0/20', '143.55.64.0/20', '2a0a:a440::/29', '2606:50c0::/32', '20.201.28.148/32', '20.205.243.168/32', '20.87.245.6/32', '20.248.137.49/32', '20.207.73.85/32', '20.27.177.116/32', '20.200.245.245/32', '20.175.192.149/32', '20.233.83.146/32', '20.29.134.17/32', '20.199.39.228/32', '4.208.26.200/32', '20.26.156.210/32', '192.30.252.0/22', '185.199.108.0/22', '140.82.112.0/20', '143.55.64.0/20', '2a0a:a440::/29', '2606:50c0::/32', '20.201.28.151/32', '20.205.243.166/32', '20.87.245.0/32', '20.248.137.48/32', '20.207.73.82/32', '20.27.177.113/32', '20.200.245.247/32', '20.175.192.147/32', '20.233.83.145/32', '20.29.134

In [182]:
# load the google endpoint json file
with open("google.json") as f:
    google_endpoints = json.load(f)

# keep only the objects that have a key "scope" that contains "northamerica"
google_endpoints_northamerica = []
for endpoint in google_endpoints["prefixes"]:
    if "scope" in endpoint and "northamerica" in endpoint["scope"].lower():
        google_endpoints_northamerica.append(endpoint)

google_cidr_blocks = []
for endpoint in google_endpoints_northamerica:
  if "ipv4Prefix" in endpoint:
    google_cidr_blocks.append(endpoint["ipv4Prefix"])
  if "ipv6Prefix" in endpoint:
    google_cidr_blocks.append(endpoint["ipv6Prefix"])


#

[{'ipv4Prefix': '34.19.128.0/17', 'service': 'Google Cloud', 'scope': 'northamerica-northeast1'}, {'ipv4Prefix': '34.20.0.0/17', 'service': 'Google Cloud', 'scope': 'northamerica-northeast1'}, {'ipv4Prefix': '34.47.0.0/18', 'service': 'Google Cloud', 'scope': 'northamerica-northeast1'}, {'ipv4Prefix': '34.95.0.0/18', 'service': 'Google Cloud', 'scope': 'northamerica-northeast1'}, {'ipv4Prefix': '34.104.76.0/22', 'service': 'Google Cloud', 'scope': 'northamerica-northeast1'}]
['34.19.128.0/17', '34.20.0.0/17', '34.47.0.0/18', '34.95.0.0/18', '34.104.76.0/22']


In [184]:
# Compare if the IP addresses in unique_destination_ips are in the CIDR blocks of the Github and Google endpoints
from netaddr import IPNetwork, IPAddress

# Convert CIDR blocks to IPNetwork objects
github_cidr_blocks = [IPNetwork(block) for block in github_cidr_blocks]
google_cidr_blocks = [IPNetwork(block) for block in google_cidr_blocks]

# Initialize list of matching IP addresses
matching_ips = []

# Iterate over unique IP addresses
for ip in unique_destination_ips:
  # Check if IP address is in any CIDR block
  if any(IPAddress(ip) in block for block in github_cidr_blocks + google_cidr_blocks):
    # If it is, add it to the list of matching IP addresses
    matching_ips.append(ip)

print(len(matching_ips))
print(matching_ips[:10])

print(unique_destination_ips[:5])
print(github_cidr_blocks[:5])
print(google_cidr_blocks[:5])

0
[]
['168.63.129.16' '142.251.163.121' '172.253.115.121' '67.220.246.12'
 '142.251.16.121']
[IPNetwork('192.30.252.0/22'), IPNetwork('185.199.108.0/22'), IPNetwork('140.82.112.0/20'), IPNetwork('143.55.64.0/20'), IPNetwork('2a0a:a440::/29')]
[IPNetwork('34.19.128.0/17'), IPNetwork('34.20.0.0/17'), IPNetwork('34.47.0.0/18'), IPNetwork('34.95.0.0/18'), IPNetwork('34.104.76.0/22')]


Load the cloudquery logs

In [ ]:
# Split the log entries into separate columns
cloudquery_logs_df = cloudquery_logs_df.iloc[:, 0].str.split(" ", expand=True)

In [ ]:
pd.set_option("display.max_colwidth", None)

# Display the first few rows to check the result
print(cloudquery_logs_df.head(5))

In [ ]:
import re

# Regular expression for date format 'YYYY-MM-DDTHH:MM:SSZ'
date_format = re.compile("\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z")

# Check that all entries in the first column match the date format
is_format_correct = cloudquery_logs_df[0].apply(lambda x: bool(date_format.match(x)))

# If any entries do not match the format, print them
if not is_format_correct.all():
    print(cloudquery_logs_df[~is_format_correct])



In [ ]:
# Define the regex pattern for the timestamp
pattern = r'^(\d{2}:\d{2}:\d{2}\.\d{6})'

# Extract the timestamp from the first column
connection_logs_df['timestamp'] = connection_logs_df[0].str.extract(pattern)

In [ ]:
filtered_df = connection_logs_df[connection_logs_df["timestamp"].notnull()]

In [ ]:
# Set max column width to None to display the entire content of each cell
pd.set_option("display.max_colwidth", None)

# Display the first five rows
print(filtered_df.head(5))
print(filtered_df.shape)

In [ ]:
# Define the regex pattern for the different parts of the string
pattern = r'^(?P<timestamp>\d{2}:\d{2}:\d{2}\.\d{6})\s+(?P<interface>\S+)\s+(?P<direction>\S+)\s+IP\s+(?P<ip_from>\S+)\s+>\s+(?P<ip_to>\S+):\s+(?P<info>.*)$'

# Extract the different parts from the first column
extracted_df = filtered_df[0].str.extract(pattern)

In [ ]:
print(extracted_df.head(5))

In [ ]:
# Find rows where 'timestamp' is NaN
nan_rows = extracted_df[extracted_df['timestamp'].isna()]

# Print the original data for these rows
print(filtered_df.loc[nan_rows.index, 0])